# импорты

In [25]:
import boto3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import string
import re
import nltk

from tqdm import trange
from nltk import tokenize
from nltk.corpus import stopwords


from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist
from collections import Counter

my_stopwords = [] #заглушка если придумаем стопслова
nltk.download('stopwords')
nltk.download('wordnet')
russian_stopwords_new = stopwords.words("russian")
russian_stopwords_new.extend(my_stopwords)
not_stopwords = {'не', 'ни'}
russian_stopwords = [word for word in russian_stopwords_new if word not in not_stopwords]

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, f1_score
from matplotlib import pyplot as plt

In [2]:
import sys
print(sys.path)

['/Users/dariamishina/Documents/HSE/YearProject/mlds23-authorship-identification/ML', '/Users/dariamishina/.conda/envs/untitled/lib/python38.zip', '/Users/dariamishina/.conda/envs/untitled/lib/python3.8', '/Users/dariamishina/.conda/envs/untitled/lib/python3.8/lib-dynload', '', '/Users/dariamishina/.conda/envs/untitled/lib/python3.8/site-packages']


In [3]:
# добавляем в path папку с тем файлом, откуда надо импортировать функции
sys.path.append('/Users/dariamishina/Documents/HSE/YearProject/mlds23-authorship-identification')

In [4]:
from data_preproc.src.utils import preprocess_text1, preprocess_text2, preprocess_text3, preprocess_text4, tokenizing

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dariamishina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/dariamishina/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
session = boto3.session.Session()

In [6]:
s3 = session.client(
    service_name='s3',
    endpoint_url='https://storage.yandexcloud.net',
    aws_access_key_id='YCAJErlaldUmioGbHQSqJ70MR',
    aws_secret_access_key='YCPSba_JgloNYSNWcnKO2CYCEB8PFR1Iwgr2jIUy',
    region_name='ru-cental1'
)

# Загружаем уже расспличенные тексты

In [8]:
BUCKET_NAME = "mlds23-authorship-identification"
BUCKET_DIR = "splitted_data/"
FILENAME = 'splitted_df.csv'
#загружает в локальную директорию, потом отдельно надо считывать
s3.download_file(Filename=FILENAME, Bucket=BUCKET_NAME, Key=BUCKET_DIR + FILENAME)

In [7]:
df = pd.read_csv('splitted_df.csv')

In [8]:
df.head()

,target,text,book
0,author_id_06,"Узкими горными тропинками , от одного дачного ...",raw_data/aleksandr_kuprin_belyj_pudel'.txt
1,author_id_06,свернутый ковер для акробатических упражнений ...,raw_data/aleksandr_kuprin_belyj_pudel'.txt
2,author_id_06,"позабытые . Кроме того , были в шарманке две п...",raw_data/aleksandr_kuprin_belyj_pudel'.txt
3,author_id_06,тайной грусти : — Что поделаешь ? .. Древний о...,raw_data/aleksandr_kuprin_belyj_pudel'.txt
4,author_id_06,", да уж ладно ! Кормила она нас с тобой , Серг...",raw_data/aleksandr_kuprin_belyj_pudel'.txt


#  Выделяем классы метками

In [11]:
enc = LabelEncoder()
df['author_id'] = enc.fit_transform(df['target'])

In [12]:
#посмотрим на распределение классов
df.groupby('author_id').author_id.count().sort_values()

author_id
1     379
7     446
8     627
5     725
3     804
6    1047
4    1295
0    1299
9    2387
2    2706
Name: author_id, dtype: int64

# подготовка для TF-IDF

In [19]:
#используем самую "суровую предобработку" с лемматизацией
df['text_proc1'] = df['text'].apply(lambda x: preprocess_text1(x, tostr=True))

In [21]:
df.head()

,target,text,book,author_id,text_proc1
0,author_id_06,"Узкими горными тропинками , от одного дачного ...",raw_data/aleksandr_kuprin_belyj_pudel'.txt,6,узкий горный тропинка дачный поселок пробирать...
1,author_id_06,свернутый ковер для акробатических упражнений ...,raw_data/aleksandr_kuprin_belyj_pudel'.txt,6,свернутый ковер акробатический упражнение прав...
2,author_id_06,"позабытые . Кроме того , были в шарманке две п...",raw_data/aleksandr_kuprin_belyj_pudel'.txt,6,позабывать кроме шарманка предательский труба ...
3,author_id_06,тайной грусти : — Что поделаешь ? .. Древний о...,raw_data/aleksandr_kuprin_belyj_pudel'.txt,6,тайный грусть — поделать древний орган прос...
4,author_id_06,", да уж ладно ! Кормила она нас с тобой , Серг...",raw_data/aleksandr_kuprin_belyj_pudel'.txt,6,ладно кормить сергей сей пора бог давать покор...


# TF-IDF

In [45]:
x_train, x_test, y_train, y_test = train_test_split(df['text_proc1'], df['author_id'], test_size = 0.3, random_state=42,\
                                                    stratify=df['author_id'])

In [46]:
def tfidf_model(x_train, y_train):
    
    tf_idf = TfidfVectorizer(stop_words=russian_stopwords, max_features=550, min_df=2)
    
    logit = LogisticRegression(C=1e2, n_jobs=4, solver='sag', #solver='lbfgs', #проблема с этим солвером
                           random_state=42, verbose=0, 
                           multi_class='multinomial',
                           fit_intercept=True, max_iter=400)
   
    tfidf_logit_pipeline = Pipeline([('tf_idf', tf_idf), 
                                 ('logit', logit)])
    tfidf_logit_pipeline.fit(x_train, y_train)
    return tfidf_logit_pipeline

In [47]:
tfidf_logit_pipeline = tfidf_model(x_train, y_train)

In [48]:
%%time
y_pred = tfidf_logit_pipeline.predict(x_test)

CPU times: user 151 ms, sys: 7.26 ms, total: 158 ms
Wall time: 188 ms


In [49]:
f1_score(y_test, y_pred, average='macro')

0.590514812375762

In [50]:
f1_score(y_test, y_pred, average='micro')

0.6364153627311522

In [51]:
f1_score(y_test, y_pred, average='weighted')

0.6364694242915129